# 1 - Relational Data Models

## ToDo

1. Breakdown into BCNF & display lossless retrieval
2. Do op's covered in complex_queries to reformatted database

Things to look into (potenitally relevant for sprint)
- Closure algo (req'd): Discover FD's ***
- Projection algo: 'Projecting' FD's onto subsets of R (used for decomp?)
- Decomp algo: Eliminates anomalies/risk of errors when working w db (req'd)
- Chase test: Test to ensure lossless decomp.

Should do this for all tables in db

### 1. BCNF

- Functional dependencies
- Closure
- Projection of functional dependences
- Keys, superkeys
- BCNF
- Lossless joins
- Relational algebra; express constraints in terms of relational algebra

### Functional Dependencies

#### For Main Tables:
- `SessionID` $\rightarrow$ `SessionID`
- `PortfolioID` $\rightarrow$ `PortfolioID`, `TotalAmt`, `Risk`
- `AllocID` $\rightarrow$ `AllocID`, `Ticker`, `Amount`
- `StockID` $\rightarrow$ `StockID`, `Ticker`, `Sector`, `Price`, `SD`, `ERet`
- `HistoryID` $\rightarrow$ `HistoryID`, `Ticker`, `Date`, `Price`
- (`PortfolioID`, `StockID`) $\rightarrow$ `PortfolioID`, `StockID`
- (`AllocID`, `StockID`) $\rightarrow$ `AllocID`, `StockID`
- (`StockID`, `HistoryID`) $\rightarrow$ `StockID`, `HistoryID`
- (`SessionID`, `PortfolioID`) $\rightarrow$ `SessionID`, `PortfolioID`
- (`PortfolioID`, `AllocID`) $\rightarrow$ `PortfolioID`, `AllocID`

This concise summary lists all functional dependencies derived from your table definitions and their respective primary keys.

### 2. Complex queries

- Go over selection of examples from class; include relational algebra which describes each operation

In [ ]:
import yfinance as yf

# Function to fetch stock data
def fetch_stock_data(ticker, stock_id):
    try:
        # Create a Ticker object for the specified ticker
        stock = yf.Ticker(ticker)
        
        # Fetch stock info with safe default values
        stock_info = stock.info
        symbol = stock_info.get('symbol', ticker)  # Default to the input ticker if 'symbol' is not available
        sector = stock_info.get('sector', 'Unknown')  # Default to 'Unknown' if 'sector' is not available
        price = stock_info.get('regularMarketPreviousClose', 0.0)  # Default to 0.0 if 'regularMarketPreviousClose' is not available
        sd = stock_info.get('beta', 0.0)  # Default to 0.0 if 'beta' is not available
        eret = stock_info.get('forwardEps', 0.0)  # Default to 0.0 if 'forwardEps' is not available
        
        return stock_id, symbol, sector, price, sd, eret
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return stock_id, None, None, None, None, None

# Prompt the user for stock ticker symbols
tickers = input("Enter the stock ticker symbols (separated by commas): ").split(',')

# Initialize stock_id counter
stock_id = 1

# List to hold the fetched data
stock_data_list = []

# Fetch and store stock data for each ticker symbol
for ticker in tickers:
    ticker = ticker.strip()  # Remove any leading/trailing whitespace
    stock_data = fetch_stock_data(ticker, stock_id)
    if stock_data[1]:  # Check if symbol is not None
        stock_data_list.append(stock_data)
    stock_id += 1

# Prepare the SQL insert statements
insert_statements = []
for data in stock_data_list:
    stock_id, symbol, sector, price, sd, eret = data
    insert_statement = f"INSERT INTO stock (StockID, Ticker, Sector, Price, SD, ERet) VALUES ({stock_id}, '{symbol}', '{sector}', {price}, {sd}, {eret});"
    insert_statements.append(insert_statement)

# Combine all insert statements into a single string
sql_insert_statements = "\n".join(insert_statements)

# Save the SQL insert statements to a file
with open("insert_statements.sql", "w") as file:
    file.write(sql_insert_statements)

# Display the SQL insert statements
print(sql_insert_statements)
